### Иерархическая индексация

In [1]:
import pandas as pd
import numpy as np

#### Мультииндексированный Series

*Плохой способ*

In [2]:
index = [('California', 2000), ('California', 2010), 
         ('New York', 2000), ('New York', 2010), 
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
              18976457, 19378102,
              20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [3]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [4]:
# фильтрация будет сложной
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

*Лучший способ*

In [5]:
index = pd.MultiIndex.from_tuples(index) # проиндексируем все
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [6]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [7]:
# срез
pop[:, 2010] # строки все, столбцы только с 2010

California    37253956
New York      19378102
Texas         25145561
dtype: int64

#### Мультииндекс как дополнительное измерение

In [8]:
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [9]:
pop.unstack() # преобразует мультииндексированный Series в DataFrame

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [10]:
pop_df = pop.unstack()

In [11]:
pop_df.stack() # обратно

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [12]:
# добавим столбец

In [13]:
pop_df = pd.DataFrame({'total':pop, 
                      'under18': [9267089, 9284094,
                                 4687374, 4318033, 
                                 5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [14]:
# вычислим доли населения до 18 лет
f_u18 = pop_df['under18'] / pop_df['total']
f_u18

California  2000    0.273594
            2010    0.249211
New York    2000    0.247010
            2010    0.222831
Texas       2000    0.283251
            2010    0.273568
dtype: float64

In [15]:
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


#### Методы создания мультииндексов

In [16]:
df = pd.DataFrame(np.random.rand(4,2),
                 index = [['a','a','b','b'], [1,2,1,2]],
                 columns = ['data1', 'data2'])
df

data1     data2
a 1  0.499321  0.493098
  2  0.231806  0.658638
b 1  0.239845  0.403675
  2  0.141679  0.603142

In [17]:
# Pandas понимает такой интаксис
data = {
    ('California', 2000):33871648,
    ('California', 2010):37253956,
    ('New York', 2000):18976457,
    ('New York', 2010):19378102,
    ('Texas', 2000):20851820,
    ('Texas', 2010):25145561
}

pd.Series(data)

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

#### Явные конструкторы для MultiIndex

In [18]:
# из простого списка массивов
pd.MultiIndex.from_arrays([['a','a','b','b'], [1,2,1,2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [19]:
# из списка кортежей
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [20]:
# из декартова произведения обычных индексов
pd.MultiIndex.from_product([['a','b'], [1,2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [21]:
# непосредственно с помощью внутреннего представления
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

#### Названия уровней мультииндексов

In [22]:
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [23]:
pop.index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [24]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

#### Мультииндекс для столбцов

In [25]:
# иерархические индексы для столбцов
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]], 
                                  names = ['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['heart_rate', 'Temp']], 
                                    names = ['subject', 'type'])

In [26]:
# создаем имитационные данные
data = np.round(np.random.randn(4,6), 1) #np.round - Округление массива до заданного числа десятичных дробей
data

array([[-0.1, -0.1,  0.6, -0.5, -0.6,  0.5],
       [-0.6, -1.6, -1.2,  1.4,  0.7,  0.1],
       [ 0.7, -0.6,  0.2, -1.7,  0.4, -1.5],
       [ 0.4,  1.1, -1.1, -0.1, -1.7,  1.1]])

In [27]:
data[:, ::2] *= 10
data += 37
data

array([[36. , 36.9, 43. , 36.5, 31. , 37.5],
       [31. , 35.4, 25. , 38.4, 44. , 37.1],
       [44. , 36.4, 39. , 35.3, 41. , 35.5],
       [41. , 38.1, 26. , 36.9, 20. , 38.1]])

In [28]:
# создаем DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject           Bob            Guido              Sue      
type       heart_rate  Temp heart_rate  Temp heart_rate  Temp
year visit                                                   
2013 1           36.0  36.9       43.0  36.5       31.0  37.5
     2           31.0  35.4       25.0  38.4       44.0  37.1
2014 1           44.0  36.4       39.0  35.3       41.0  35.5
     2           41.0  38.1       26.0  36.9       20.0  38.1

### Индексация и срезы по мультииндексу

#### Мультииндексация в Series

In [29]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [30]:
pop['California', 2010]

37253956

In [31]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [32]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [33]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [34]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [35]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

#### Мультииндексация в DataFrame

In [36]:
health_data

subject           Bob            Guido              Sue      
type       heart_rate  Temp heart_rate  Temp heart_rate  Temp
year visit                                                   
2013 1           36.0  36.9       43.0  36.5       31.0  37.5
     2           31.0  35.4       25.0  38.4       44.0  37.1
2014 1           44.0  36.4       39.0  35.3       41.0  35.5
     2           41.0  38.1       26.0  36.9       20.0  38.1

In [37]:
health_data['Guido', 'heart_rate']

year  visit
2013  1        43.0
      2        25.0
2014  1        39.0
      2        26.0
Name: (Guido, heart_rate), dtype: float64

In [38]:
health_data.iloc[:2, :2]

subject           Bob      
type       heart_rate  Temp
year visit                 
2013 1           36.0  36.9
     2           31.0  35.4

In [39]:
health_data.loc[:, ('Guido', 'heart_rate')]

year  visit
2013  1        43.0
      2        25.0
2014  1        39.0
      2        26.0
Name: (Guido, heart_rate), dtype: float64

In [40]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'heart_rate']] # idx[все года, визит только 1], idx[все имена, показатель только 'heart_rate']

,subject,Bob,Guido,Sue
,type,heart_rate,heart_rate,heart_rate
year,visit,,,
2013,1,36.0,43.0,31.0
2014,1,44.0,39.0,41.0


### Перегруппировка мультииндексов

#### Отсортированные и неотсортированные индексы

In [41]:
# большинство операций срезов с неотсортированными индексами приведут к ошибке
index = pd.MultiIndex.from_product([['a','c','b'], [1,2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names= ['char', 'int']
data

char  int
a     1      0.212291
      2      0.692912
c     1      0.997415
      2      0.134905
b     1      0.682735
      2      0.131079
dtype: float64

In [42]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


In [43]:
data = data.sort_index()
data

char  int
a     1      0.212291
      2      0.692912
b     1      0.682735
      2      0.131079
c     1      0.997415
      2      0.134905
dtype: float64

In [44]:
data['a':'b']

char  int
a     1      0.212291
      2      0.692912
b     1      0.682735
      2      0.131079
dtype: float64

#### Выполнение над индексами операций stack и unstack

In [45]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [46]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [47]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [48]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

#### Создание и перестройка индексов

In [49]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [50]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [51]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

### Агрегирование по мультииндексам

In [52]:
health_data

subject           Bob            Guido              Sue      
type       heart_rate  Temp heart_rate  Temp heart_rate  Temp
year visit                                                   
2013 1           36.0  36.9       43.0  36.5       31.0  37.5
     2           31.0  35.4       25.0  38.4       44.0  37.1
2014 1           44.0  36.4       39.0  35.3       41.0  35.5
     2           41.0  38.1       26.0  36.9       20.0  38.1

In [53]:
data_mean=health_data.mean(level='year')
data_mean

subject        Bob             Guido               Sue      
type    heart_rate   Temp heart_rate   Temp heart_rate  Temp
year                                                        
2013          33.5  36.15       34.0  37.45       37.5  37.3
2014          42.5  37.25       32.5  36.10       30.5  36.8

In [54]:
data_mean.mean(axis=1, level='type')

type,heart_rate,Temp
year,,
2013,35.000000,36.966667
2014,35.166667,36.716667


### Объединение наборов данных: конкатенация и добавление в конец

In [55]:
def make_df(cols, ind):
    """Быстрое создание датафреймов"""
    data = {c: [str(c) + str(i) for i in ind] for c in cols}
    return pd.DataFrame(data, ind)

In [56]:
make_df('ABC', range(5))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2
3,A3,B3,C3
4,A4,B4,C4


#### Напоминание: конкатенация массивов NumPy

In [57]:
x = [1,2,3]
y = [4,5,6]
z = [7,8,9]
np.concatenate([x,y,z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [58]:
x = [[1,2],
     [3,4]]
np.concatenate([x,x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

In [59]:
x = [[1,2],
     [3,4]]
np.concatenate([x,x], axis=0)

array([[1, 2],
       [3, 4],
       [1, 2],
       [3, 4]])

### Простая конкатенация метод pd.concat

In [60]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1,2,3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4,5,6])
pd.concat([ser1, ser2]) # axis=0 по умолчанию (построчно)

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

In [61]:
df1 = make_df('AB', [1,2])
df2 = make_df('AB', [3,4])
print(df1)
print()
print(df2)
print()
print(pd.concat([df1, df2]))

    A   B
1  A1  B1
2  A2  B2

    A   B
3  A3  B3
4  A4  B4

    A   B
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4


In [62]:
df3 = make_df('AB', [0,1])
df4 = make_df('CD', [0,1])
print(df3)
print()
print(df4)
print()
print(pd.concat([df3, df4], axis='columns')) # axis=1

    A   B
0  A0  B0
1  A1  B1

    C   D
0  C0  D0
1  C1  D1

    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1


#### Дублирование индексов

In [63]:
x = make_df('AB', [0,1])
y = make_df('AB', [2,3])
y.index = x.index # задублируем индексы
print(x)
print()
print(y)
print()
print(pd.concat([x, y]))

    A   B
0  A0  B0
1  A1  B1

    A   B
0  A2  B2
1  A3  B3

    A   B
0  A0  B0
1  A1  B1
0  A2  B2
1  A3  B3


Повтор индексов может быть нежелателен

##### Перехват повторов как ошибки

In [64]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError: ", e)

ValueError:  Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


##### Игнорирование индекса

Вместо повторов создается индекс по порядку

In [65]:
x = make_df('AB', [0,1])
y = make_df('AB', [2,3])
y.index = x.index 
print(x)
print()
print(y)
print()
print(pd.concat([x, y], ignore_index=True))

    A   B
0  A0  B0
1  A1  B1

    A   B
0  A2  B2
1  A3  B3

    A   B
0  A0  B0
1  A1  B1
2  A2  B2
3  A3  B3


##### Добаавление ключей мультииндекса

In [66]:
x = make_df('AB', [0,1])
y = make_df('AB', [2,3])
y.index = x.index 
print(x)
print()
print(y)
print()
print(pd.concat([x, y], keys = ['first', 'second']))

    A   B
0  A0  B0
1  A1  B1

    A   B
0  A2  B2
1  A3  B3

           A   B
first  0  A0  B0
       1  A1  B1
second 0  A2  B2
       1  A3  B3


### Конкатенация с использованием соединений

In [67]:
import sys
df5 = make_df('ABC', [1,2])
df6 = make_df('BCD', [3,4])
print(df5)
print()
print(df6)
print()
print(pd.concat([df5, df6]))

    A   B   C
1  A1  B1  C1
2  A2  B2  C2

    B   C   D
3  B3  C3  D3
4  B4  C4  D4

     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


По умолчанию происходит outer объединение

Можно сделать inner

In [68]:
df5 = make_df('ABC', [1,2])
df6 = make_df('BCD', [3,4])
print(df5)
print()
print(df6)
print()
print(pd.concat([df5, df6], join='inner'))

    A   B   C
1  A1  B1  C1
2  A2  B2  C2

    B   C   D
3  B3  C3  D3
4  B4  C4  D4

    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4


Параметр join_axes помогает установить, столбцы какого из датафреймов нельзя терять

In [69]:
df5 = make_df('ABC', [1,2])
df6 = make_df('BCD', [3,4])
print(df5)
print()
print(df6)
print()
print(pd.concat([df5, df6], join_axes=[df5.columns]))

    A   B   C
1  A1  B1  C1
2  A2  B2  C2

    B   C   D
3  B3  C3  D3
4  B4  C4  D4

     A   B   C
1   A1  B1  C1
2   A2  B2  C2
3  NaN  B3  C3
4  NaN  B4  C4


#### Метод append()

pd.concat([df1, df2]) соответствует df1.append(df2)

In [70]:
print(df1)
print()
print(df2)
print()
print(df1.append(df2))

    A   B
1  A1  B1
2  A2  B2

    A   B
3  A3  B3
4  A4  B4

    A   B
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4


append - создает новый объект

pd.concat - нет

### Объединение наборов данных: слияние и соединение

#### Виды соединений

#### Один к одному

In [71]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
print(df1)
print()
print(df2)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014


In [72]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


Функция merge распознает, что есть одинаковый столбец 'employee' и соединяет по нему, как по ключу

#### Многие к одному

In [73]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                   'supervisor': ['Carly', 'Guido', 'Steve']})
print(df3)
print()
print(df4)
print()
print(pd.merge(df3, df4))

  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve


#### Многие ко многим

In [74]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting', 'Engineering', 'Engineering', 'HR', 'HR'],
                   'skills': ['math', 'spreadsheets', 'coding', 'linux', 'spreadsheets','organization']})
print(df1)
print()
print(df5)
print()
print(pd.merge(df1, df5))

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

         group        skills
0   Accounting          math
1   Accounting  spreadsheets
2  Engineering        coding
3  Engineering         linux
4           HR  spreadsheets
5           HR  organization

  employee        group        skills
0      Bob   Accounting          math
1      Bob   Accounting  spreadsheets
2     Jake  Engineering        coding
3     Jake  Engineering         linux
4     Lisa  Engineering        coding
5     Lisa  Engineering         linux
6      Sue           HR  spreadsheets
7      Sue           HR  organization


#### Задание ключа слияния

#### Ключевое слово on

Применяем, когда названия столбцов совпадают

In [75]:
print(df1)
print()
print(df2)
print()
print(pd.merge(df1, df2, on='employee'))

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014


#### Ключевые слова left_on и right_on

Применяем, когда названия столбцов не совпадают

In [76]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                   'salary': [70000, 80000, 120000, 90000]})

In [77]:
print(df1)
print()
print(df3)
print()
print(pd.merge(df1, df3, left_on='employee', right_on='name'))

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000


In [78]:
# можно сразу удалить избыточный столбец
pd.merge(df1, df3, left_on='employee', right_on='name').drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


#### Ключевые слова left_index и right_index

Когда нужно слияние по индексу

In [79]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
print(df1a)
print()
print(df2a)

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014


In [80]:
print(df1a)
print()
print(df2a)
print()
print(pd.merge(df1a, df2a, left_index=True, right_index=True))

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014

                group  hire_date
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


In [81]:
# то же самое методом join
print(df1a)
print()
print(df2a)
print()
print(df1a.join(df2a))

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014

                group  hire_date
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


In [82]:
# комбинация способов
print(df1a)
print()
print(df3)
print()
print(pd.merge(df1a, df3, left_index=True, right_on='name'))

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000


### Задание операций над множествами для соединений

Когда есть отсутствующие значения

In [83]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                   'food': ['fish', 'beans', 'bread']}, columns = ['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                   'drink': ['wine', 'beer']}, columns = ['name', 'drink'])
print(df6)
print()
print(df7)
print()
print(pd.merge(df6, df7)) # по умолчанию inner join

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

     name drink
0    Mary  wine
1  Joseph  beer

   name   food drink
0  Mary  bread  wine


Inner

In [84]:
pd.merge(df6, df7, how = 'inner') # явно указываем вид соединения

,name,food,drink
0,Mary,bread,wine


Outer

In [85]:
print(df6)
print()
print(df7)
print()
print(pd.merge(df6, df7, how = 'outer'))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

     name drink
0    Mary  wine
1  Joseph  beer

     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer


Left

In [86]:
print(df6)
print()
print(df7)
print()
print(pd.merge(df6, df7, how = 'left'))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

     name drink
0    Mary  wine
1  Joseph  beer

    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine


Right

In [87]:
print(df6)
print()
print(df7)
print()
print(pd.merge(df6, df7, how = 'right'))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

     name drink
0    Mary  wine
1  Joseph  beer

     name   food drink
0    Mary  bread  wine
1  Joseph    NaN  beer


### Пересекающиеся названия столбцов: ключевое слово suffixes

Конфликтующие названия столбцов

In [88]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                   'rank': [1,2,3,4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                   'rank': [3,1,4,2]})
print(df8)
print()
print(df9)
print()
print(pd.merge(df8, df9, on = 'name'))

   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


In [89]:
# Выше функция добавила в названия суффиксы. Можно их задать
print(pd.merge(df8, df9, on = 'name', suffixes=['_L', '_R']))

   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


### Пример: данные по штатам США

In [90]:
pop = pd.read_csv('state-population.csv')
areas = pd.read_csv('state-areas.csv')
abbrevs = pd.read_csv('state-abbrevs.csv')

In [91]:
print(pop.head())
print()
print(areas.head())
print()
print(abbrevs.head())

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


In [92]:
merged = pd.merge(pop, abbrevs, how = 'outer',
                 left_on = 'state/region', right_on = 'abbreviation')
merged.head()

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL


In [93]:
merged = merged.drop('abbreviation', 1)
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [94]:
# проверим на строки с путыми значениями. Если они есть, значит были несовпадения
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [95]:
# часть информации отсутствует. Узнаем, какая.
#с помощью isnull можно найти пустые (NA) значения
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


In [96]:
# по пуэрто-Рико много пропущенных данных + не по всем штатам есть аббревиатура
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [97]:
# Теперь ясно: не указаны названия для аббревиатур PR и USA (в целом)
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'


In [98]:
# пустые значения остались только в population       
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [99]:
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [100]:
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [101]:
# сольем результат с данными по площади - по "state"
final = pd.merge(merged, areas, on = 'state', how = 'left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [102]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [103]:
# в толбце 'area' есть пустые значения
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

In [104]:
# удалим данные по США в целом
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [105]:
delta = len(final) / len(pop) * 100
print("Размер очищенного датафрейма относительно начального: ", round(delta, 2), "%")

Размер очищенного датафрейма относительно начального:  97.33 %


In [106]:
# исследуем данные за 2010 год
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


In [107]:
# Вычислим плотность населения и изменим порядок вывода данных
data2010.set_index('state', inplace=True)
data2010.head()

,state/region,ages,year,population,area (sq. mi)
state,,,,,
Alabama,AL,total,2010,4785570.0,52423.0
Alaska,AK,total,2010,713868.0,656425.0
Arizona,AZ,total,2010,6408790.0,114006.0
Arkansas,AR,total,2010,2922280.0,53182.0
California,CA,total,2010,37333601.0,163707.0


In [108]:
density = data2010['population'] / data2010['area (sq. mi)']
density

state
Alabama                   91.287603
Alaska                     1.087509
Arizona                   56.214497
Arkansas                  54.948667
California               228.051342
Colorado                  48.493718
Connecticut              645.600649
Delaware                 460.445752
District of Columbia    8898.897059
Florida                  286.597129
Georgia                  163.409902
Hawaii                   124.746707
Idaho                     18.794338
Illinois                 221.687472
Indiana                  178.197831
Iowa                      54.202751
Kansas                    34.745266
Kentucky                 107.586994
Louisiana                 87.676099
Maine                     37.509990
Maryland                 466.445797
Massachusetts            621.815538
Michigan                 102.015794
Minnesota                 61.078373
Mississippi               61.321530
Missouri                  86.015622
Montana                    6.736171
Nebraska              

In [109]:
density.sort_values(ascending=False, inplace=True) # ascending=False значит по убыванию

In [110]:
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [111]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

### Агрегирование и группировка

#### Данные о планетах

In [112]:
import seaborn as sns # качаем встроенный датасет сиборна
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [113]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


#### Простое агрегирование

In [114]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [115]:
ser.sum()

2.811925491708157

In [116]:
ser.mean()

0.5623850983416314

In [117]:
# в случае с датафреймом агрегирующие функции считают по каждому столбцу по умолчанию
df = pd.DataFrame({'A': rng.rand(5),
                  'B': rng.rand(5)})
df

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


In [118]:
df.mean()

A    0.477888
B    0.443420
dtype: float64

In [119]:
# агрегируем по строкам 
df.mean(axis = 1)

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

In [120]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


### GroupBy: разбиение, применение, объединение

In [121]:
df = pd.DataFrame({'key': ['A','B','C','A','B','C'],
                  'data': range(1,7)}, columns=['key', 'data'])
df

,key,data
0,A,1
1,B,2
2,C,3
3,A,4
4,B,5
5,C,6


In [122]:
df.groupby('key')

In [123]:
df.groupby('key').sum()

,data
key,
A,5
B,7
C,9


#### Объект GroupBy

##### Индексация по столбцам

In [124]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [125]:
planets.groupby('method')['orbital_period'].median() # сгруппируй по методам и выдай медиану по периодам обращения в каждом методе

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

##### Цикл по группам

In [126]:
for (method, group) in planets.groupby('method'):
    print("{0:29s} shape={1}".format(method, group.shape)) # 29s - длина строки, добавляется недостающее количество пропусков

Astrometry                    shape=(2, 6)
Eclipse Timing Variations     shape=(9, 6)
Imaging                       shape=(38, 6)
Microlensing                  shape=(23, 6)
Orbital Brightness Modulation shape=(3, 6)
Pulsar Timing                 shape=(5, 6)
Pulsation Timing Variations   shape=(1, 6)
Radial Velocity               shape=(553, 6)
Transit                       shape=(397, 6)
Transit Timing Variations     shape=(4, 6)


##### Методы диспетчеризации

In [127]:
planets.groupby('method')['year'].describe()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Astrometry,2.0,2011.500000,2.121320,2010.0,2010.75,2011.5,2012.25,2013.0
Eclipse Timing Variations,9.0,2010.000000,1.414214,2008.0,2009.00,2010.0,2011.00,2012.0
Imaging,38.0,2009.131579,2.781901,2004.0,2008.00,2009.0,2011.00,2013.0
Microlensing,23.0,2009.782609,2.859697,2004.0,2008.00,2010.0,2012.00,2013.0
Orbital Brightness Modulation,3.0,2011.666667,1.154701,2011.0,2011.00,2011.0,2012.00,2013.0
Pulsar Timing,5.0,1998.400000,8.384510,1992.0,1992.00,1994.0,2003.00,2011.0
Pulsation Timing Variations,1.0,2007.000000,NaN,2007.0,2007.00,2007.0,2007.00,2007.0
Radial Velocity,553.0,2007.518987,4.249052,1989.0,2005.00,2009.0,2011.00,2014.0
Transit,397.0,2011.236776,2.077867,2002.0,2010.00,2012.0,2013.00,2014.0


In [128]:
planets.groupby('method')['year'].describe().unstack()

       method                       
count  Astrometry                          2.000000
       Eclipse Timing Variations           9.000000
       Imaging                            38.000000
       Microlensing                       23.000000
       Orbital Brightness Modulation       3.000000
       Pulsar Timing                       5.000000
       Pulsation Timing Variations         1.000000
       Radial Velocity                   553.000000
       Transit                           397.000000
       Transit Timing Variations           4.000000
mean   Astrometry                       2011.500000
       Eclipse Timing Variations        2010.000000
       Imaging                          2009.131579
       Microlensing                     2009.782609
       Orbital Brightness Modulation    2011.666667
       Pulsar Timing                    1998.400000
       Pulsation Timing Variations      2007.000000
       Radial Velocity                  2007.518987
       Transit             

#### Агрегирование, фильтрация, преобразование, применение

In [135]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A','B','C','A','B','C'],
                  'data1': range(6),
                  'data2': rng.randint(0,10,6)}, # 6 чисел от 0 до 10
                 columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


##### Агрегирование

In [139]:
# Группируем например по ключу и передаем списком требуеме функции для вычисления
df.groupby('key').aggregate(['min', np.median, 'max'])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

In [140]:
df.groupby('key').aggregate(['min', 'median', 'max'])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

In [141]:
# можно передать словарем, какую функцию применить к какому столбцу
df.groupby('key').aggregate({'data1': 'min',
                            'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


##### Фильтрация

Когда нужно оставить в результате данные в зависимости от свойств их группы.

Например, можем оставить группы, у которых ст. ошибка превышает какое-то значение:

In [142]:
def filter_func(x):
    return x['data2'].std()>4

In [144]:
filter_func(df)

False

In [145]:
print(df)
print()
print(df.groupby('key').std())
print()
print(df.groupby('key').filter(filter_func))

  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641

  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9


##### Преобразование

Центрирование данных путем вычитания среднего по группам

In [146]:
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


##### Метод apply

Позволяет применять проивольную функцию к результатам группировки

In [147]:
def norm_by_data2(x):
    """нормирует первый столбец на сумму значений второго"""
    # x - объект DataFrame сгруппированных значений
    x['data1'] = x['data1'] / x['data2'].sum()
    #x['data1'] /= x['data2'].sum()
    return x

In [148]:
print(df)
print()
print(df.groupby('key').apply(norm_by_data2))

  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

  key     data1  data2
0   A  0.000000      5
1   B  0.142857      0
2   C  0.166667      3
3   A  0.375000      3
4   B  0.571429      7
5   C  0.416667      9


#### Задание ключа разбиения

##### Список, массив, объект Series и индекс как ключи группировки

In [156]:
L = [0,1,0,1,2,0]
print(df)
print()
print(df.groupby(L).sum())

  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

   data1  data2
0      7     17
1      4      3
2      4      7


In [157]:
print(df.groupby('key').sum())

     data1  data2
key              
A        3      8
B        5      7
C        7     12


In [158]:
df['L'] = [0,1,0,1,2,0]

In [159]:
df

,key,data1,data2,L
0,A,0,5,0
1,B,1,0,1
2,C,2,3,0
3,A,3,3,1
4,B,4,7,2
5,C,5,9,0


In [160]:
print(df.groupby(L).sum())

   data1  data2  L
0      7     17  0
1      4      3  2
2      4      7  2


In [165]:
df.drop('L', axis=1, inplace=True)
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [166]:
print(df)
print()
print(df.groupby(df['key']).sum())

  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

     data1  data2
key              
A        3      8
B        5      7
C        7     12


##### Словарь или объект Series, связывающий индекс и группу

In [167]:
df2 = df.set_index('key')
df2

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9


In [169]:
mapping = {'A': 'гласная', 'B': 'согласная', 'C': 'согласная'}
print(df2)
print()
print(df2.groupby(mapping).sum())

     data1  data2
key              
A        0      5
B        1      0
C        2      3
A        3      3
B        4      7
C        5      9

           data1  data2
гласная        3      8
согласная     12     19


##### Любая функция языка Python

In [173]:
# принимает на входе значение индекса и возвращает группу
df2.groupby(str.lower).mean()

,data1,data2
a,1.5,4.0
b,2.5,3.5
c,3.5,6.0


##### Список допустимых ключей

Можно комбинировать описанные выше варианты для группировки по мультииндексу

In [175]:
df2.groupby([str.lower, mapping]).mean()

,,data1,data2
a,гласная,1.5,4.0
b,согласная,2.5,3.5
c,согласная,3.5,6.0


#### Пример группировки

Количество открытых планет по методу открытия и десятилетию

In [192]:
planets.tail()

,method,number,orbital_period,mass,distance,year
1030,Transit,1,3.941507,NaN,172.0,2006
1031,Transit,1,2.615864,NaN,148.0,2007
1032,Transit,1,3.191524,NaN,174.0,2007
1033,Transit,1,4.125083,NaN,293.0,2008
1034,Transit,1,4.187757,NaN,260.0,2008


In [193]:
decade = 10 * (planets['year']//10) # создаем послежовательность из названий десятилетий
decade = decade.astype(str) + '-ые'

In [194]:
planets.groupby(['method', decade])['number'].sum() # сгруппируй по столбцу 'method' и по послежовательности "Десятилетия"
# возьми по ним сумму открытий, информация о которых находится в столбце 'number' 

method                         year   
Astrometry                     2010-ые      2
Eclipse Timing Variations      2000-ые      5
                               2010-ые     10
Imaging                        2000-ые     29
                               2010-ые     21
Microlensing                   2000-ые     12
                               2010-ые     15
Orbital Brightness Modulation  2010-ые      5
Pulsar Timing                  1990-ые      9
                               2000-ые      1
                               2010-ые      1
Pulsation Timing Variations    2000-ые      1
Radial Velocity                1980-ые      1
                               1990-ые     52
                               2000-ые    475
                               2010-ые    424
Transit                        2000-ые     64
                               2010-ые    712
Transit Timing Variations      2010-ые      9
Name: number, dtype: int64

In [197]:
decade.name = 'Десятилетие' #
planets.groupby(['method', decade])['number'].sum().unstack().fillna('без открытий')

Десятилетие,1980-ые,1990-ые,2000-ые,2010-ые
method,,,,
Astrometry,без открытий,без открытий,без открытий,2
Eclipse Timing Variations,без открытий,без открытий,5,10
Imaging,без открытий,без открытий,29,21
Microlensing,без открытий,без открытий,12,15
Orbital Brightness Modulation,без открытий,без открытий,без открытий,5
Pulsar Timing,без открытий,9,1,1
Pulsation Timing Variations,без открытий,без открытий,1,без открытий
Radial Velocity,1,52,475,424
Transit,без открытий,без открытий,64,712
